# Organ exchange problem
[![colab.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master//authors/mikhail/miscellaneous/organ_exchange.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/authors/mikhail/miscellaneous/organ_exchange.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master//authors/mikhail/miscellaneous/organ_exchange.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/authors/mikhail/miscellaneous/organ_exchange.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/authors/mikhail/miscellaneous/organ_exchange.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/authors/mikhail/miscellaneous/organ_exchange.ipynb)](https://colab.ampl.com)

Description: Organ exchange model

Tags: mip

Model author: ???

Notebook author: Mikhail <<mail@solverytic.com>>

References: 

In [7]:
# Install dependencies
!pip install -q amplpy

In [8]:
# Google Colab & Kaggle integration
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["highs"], # modules to install
    license_uuid="default", # license to use
    g=globals()) # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


## Model

In [9]:
%%ampl_eval
# Donor Exchange
reset;
### SETS 
  param N;
  set Blood;                      				# set of blood types
  set PATIENTS:={1..N} ordered;   				# set of patients
  set DONORS;                     				# set of donors
  set ALL:= PATIENTS union DONORS;
  set Transpl;                    				# type of transplant
  set BLOOD within {Blood, Blood};				# Possible options for the donation of blood type
  set PAT_DON within {PATIENTS, DONORS}; 			# donor-patient relationship
  set TRANSPL_HUM within {ALL,Transpl}; 			# type of transplant assigned to a person
  set BLOOD_HUM within {ALL, Blood}; 				# human blood type
  set PATIENTS_:= setof {(i,tr) in TRANSPL_HUM,
	(i,b) in BLOOD_HUM:i in PATIENTS}(i,tr,b); 		# patients data: blood type, type of transplant 
  set DONORS_:= setof {(j,tr) in TRANSPL_HUM, 
	(j,b) in BLOOD_HUM, (i,j) in PAT_DON:j in DONORS}(i,j,tr,b); 	# donors data: blood type, type of transplant
  set LINKS_:= setof {(ii,tr,b1) in PATIENTS_,
	(i,j,tr,b2) in DONORS_:(b1,b2) in BLOOD}(i,j,ii,tr,b1); 		# A set of options for possible assignments
  set LINKS:= setof {(ii,tr,b1) in PATIENTS_,
	(i,j,tr,b2) in DONORS_:(b1,b2) in BLOOD}(i,j,ii,tr,b1,b2);		# A set of options for possible assignments
  set PAT_2DON:= setof {(i,j1,ii,tr,b1,b2) in LINKS, 
  (i,j2,tr,b3) in DONORS_:(b1,b3) in BLOOD and (b2,b3) in BLOOD 
  and j1!=j2}(i,j1,j2,tr,b1,b2,b3);									# A set of options for possible assignments
	### You can add a tightening of compatibility (only by blood type) ###
### PARAMETERS
	param Clusters:= last(PATIENTS);
	param demand {(i,tr,b) in PATIENTS_}>=0; # amounts required
### VARIABLES
	var X {PATIENTS,1..Clusters} binary;    # All operations on the patient must be performed within the same cluster
	var Y{LINKS_, 1..Clusters} binary;      # the donor participates in only one cluster
### OBJECTIVE		
	maximize Total: sum {i in PATIENTS, c in 1..Clusters} X [i,c];    
### REQUAREMENTS
	subject to
		# for each cluster, supply - demand >=0;
	A {c in 1..Clusters,tr in Transpl, b in Blood}: sum{(i,j,ii,tr,b) in LINKS_}
	Y[i,j,ii,tr,b,c] - sum {(i,tr,b) in PATIENTS_} demand[i,tr,b]* X[i,c]>=0; # you can put =
		# a patient can only be part of one cluster
	A_1 {i in PATIENTS}: sum {c in 1..Clusters}X[i,c]<=1;
		# for each type of operation of the patient in all clusters, only one donor is assigned.
	A_2 {ii in PATIENTS, tr in Transpl}:sum{c in 1..Clusters,(i,j,ii,tr,b) in LINKS_}Y[i,j,ii,tr,b,c]<=1;
		# Limiting the number of patients in 1 cluster
	A_3{c in 1..Clusters}: sum {i in PATIENTS} X[i,c]<=3; 
		# A donor can participate in an organ transplant operation only 1 time 
	A_4 {j in DONORS, tr in Transpl}: sum{c in 1..Clusters, (i,j,ii,tr,b) in LINKS_} Y[i,j,ii,tr,b,c]<=1;
		# Limiting the number of operations in the cluster (it is assumed that after assigning a donor for transplantation, another operation is performed with the patient)
	A_5{c in 1..Clusters}: sum{(i,j,ii,tr,b) in LINKS_} Y[i,j,ii,tr,b,c]*2<=6;
	


## Data

In [10]:
%%ampl_eval
### DATA
	data;
  param N:=5;
  set Blood := O, A, B, AB;
  set DONORS := d01 d02 d03 d04 d05 d06 d07 d08 d09 d10 ; #number of donors
  set Transpl := Liver, Kidney, Lung;
  set BLOOD: O   A   B  AB := # Data on compatibility of blood types
	O          +   -   -  -
	A          +   +   -  -
	B          +   -   +  -
	AB         +   +   +  + ;
  set PAT_DON(tr): 
		1   2   3   4   5 :=
	d01 -   +   -   -   -
	d02 -   +   -   -   -   
	d03 +   -   -   -   -
	d04 +   -   -   -   -
	d05 -   -   +   -   -
	d06 -   -   -   -   +
	d07 -   -   -   +   -
	d08 -   -   -   -   +
	d09 -   -   +   -   -
	d10 +   -   -   -   -  ;
  set TRANSPL_HUM: Liver  Kidney  Lung:=
	d01              +      -       +       # This table assumes that each patient is operated 
	d02              -      -       +       # on in one approach, i.e. all the necessary 
	d03              +      -       -       # operations are performed at 1 time.
	d04              +      -       +      
	d05              -      +       -      
	d06              +      -       -      
	d07              +      +       +      
	d08              -      +       -      
	d09              +      -       -      
	d10              +      -       +      
	1                -      -       +      
	2                +      +       +      
	3                +      -       -      
	4                +      +       -      
	5                +      -       -     ; # This condition means and. You could change to or.
  set BLOOD_HUM:   O   A   B   AB:=
	d01              +   -   -   -    
	d02              -   +   -   -      
	d03              -   -   +   -      
	d04              -   +   -   -      
	d05              -   -   -   +      
	d06              -   -   -   +      
	d07              -   +   -   -      
	d08              +   -   -   -     
	d09              +   -   -   -      
	d10              -   -   -   +      
	1                +   -   -   -    
	2                -   +   -   -      
	3                -   -   -   +     
	4                -   +   -   -      
	5                +   -   -   -     ; 

param demand default 1;


## Solve with Highs

In [12]:
%%ampl_eval

### OPTIONS  
	option solver highs;
	
### SOLVE	
	solve;
	
### RESULTS	
	display BLOOD, PAT_2DON;
	display X, Y;

HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 5
36 simplex iterations
1 branching nodes
set BLOOD :=
(O,O)     (A,A)     (B,B)     (AB,A)    (AB,AB)
(A,O)     (B,O)     (AB,O)    (AB,B);

set PAT_2DON :=
(2,d02,d01,Lung,A,A,O)      (1,d10,d04,Liver,AB,AB,A)
(1,d10,d03,Liver,AB,AB,B);

X [*,*]
:   1   2   3   4   5    :=
1   0   0   0   0   1
2   1   0   0   0   0
3   0   0   0   0   1
4   0   0   1   0   0
5   0   1   0   0   0
;

Y [1,*,2,Liver,A,*] (tr)
: d04    :=
1   0
2   0
3   0
4   0
5   0

 [1,*,2,Lung,A,*] (tr)
: d04    :=
1   0
2   0
3   0
4   0
5   0

 [1,*,3,Liver,AB,*] (tr)
: d03 d04 d10    :=
1   0   0   0
2   0   0   0
3   0   0   0
4   0   0   0
5   1   0   0

 [1,*,4,Liver,A,*] (tr)
: d04    :=
1   0
2   0
3   0
4   0
5   0

 [2,*,1,Lung,O,*] (tr)
: d01    :=
1   0
2   0
3   0
4   0
5   1

 [2,*,2,Liver,A,*] (tr)
: d01    :=
1   0
2   0
3   0
4   0
5   0

 [2,*,2,Lung,A,*] (tr)
: d01 d02    :=
1   0   1
2   0   0
3   0   0
4   0   0
5   0   0

 [2,